# Solving a modular equation - Running on IonQ via Azure Quantum

## Load the libraries

In [ ]:
import qsharp
import qsharp.azure
from SolveModularEquation import FindEquationSolution
from SolveModularEquationSimplified import FindEquationSolutionSimplified

## Run the simulation from Python

In [ ]:
def littleendian_int(bits):
    bits.reverse()
    x = 0
    for bit in bits:
        x = x * 2 + bit
    return x

answerBits = FindEquationSolutionSimplified.simulate(a=1, b=2, c=4)
print(answerBits)
answer = littleendian_int(answerBits)
print("x = " + str(answer))

## Estimate the resources required to run the algorithm to solve this equation

In [ ]:
FindEquationSolutionSimplified.estimate_resources(
    b=2
)

We have 11 qubits on the IonQ system available via Azure Quantum, so this is feasible to run!

## Connect to Azure Quantum

In [ ]:
qsharp.azure.connect(
    subscription="...",
    resourceGroup="...",
    workspace="...",
    location="...")

## Submit job to IonQ simulator

In [ ]:
qsharp.azure.target("ionq.simulator")

In [ ]:
b = 2
job = qsharp.azure.submit(
    FindEquationSolutionSimplified, b=b, shots=100, jobName=f"Solving equation x + {b} = 0 (mod 4)"
)

Check job status

In [ ]:
def print_status(st):
    print(st.status + " | " + st.name + " | " + st.target + " | " +
          st.creation_time + " | " + st.end_execution_time)

job_id = job.id
status = qsharp.azure.status(job_id)
print_status(status)

Fetch job results and interpret them

In [ ]:
result = qsharp.azure.output(job_id)
print(result)

In [ ]:
import matplotlib.pyplot as plt
plt.bar(result.keys(), result.values())
plt.show()

In [ ]:
import ast

def most_frequent_key(histogram):
    # Get the most frequent key of the histogram
    mostFrequentBitsStr = max(histogram, key=histogram.get)
    # Convert the key to an array of ints
    mostFrequentBits = ast.literal_eval(mostFrequentBitsStr)
    # Convert the little endian notation into an integer - the solution of the equation
    answer = littleendian_int(mostFrequentBits)
    return answer

print("x = " + str(most_frequent_key(result)))

## Run the job on IonQ hardware

Now we are ready to run on hardware! Switch to the QPU.

In [ ]:
qsharp.azure.target("ionq.qpu")

Submitting the jobs looks exactly like it does for the simulator.

In [ ]:
job_hw = qsharp.azure.submit(
    FindEquationSolutionSimplified, b=b, shots=1000, jobName=f"Solving equation x + {b} = 0 (mod 4) on hardware"
)

For this demo, we'll use the results of a job submitted to the hardware when preparing the demo, rather than submit live.

In [ ]:
job_id_hw = '...'
status_hw = qsharp.azure.status(job_id_hw)
print_status(status_hw)

Get the result

In [ ]:
result_hw = qsharp.azure.output(job_id_hw)
print(result_hw)

In [ ]:
plt.bar(result_hw.keys(), result_hw.values())
plt.show()

In [ ]:
print("x = " + str(most_frequent_key(result_hw)))